Imports


In [1]:
from ultralytics import YOLO
# from trl import SFTTrainer, SFTConfig
# from peft import LoraConfig, get_peft_model
import warnings
import gc
import os
import re
import torch
# import pandas as pd
# import numpy as np

Disabling Warnings


In [2]:
warnings.filterwarnings("ignore")

Setting Training Device


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if str(device) == "cuda":
    print(f"Using GPU {torch.cuda.get_device_properties(device)}")
else:
    print("Using CPU")

Using GPU _CudaDeviceProperties(name='AMD Radeon RX 7800 XT', major=11, minor=0, gcnArchName='gfx1100', total_memory=16368MB, multi_processor_count=30, uuid=38336232-6265-3432-3662-306564613532, L2_cache_size=4MB)


Emptying the GPU Cache (if necessary)


In [4]:
def empty_cache() -> None:
    # Cleaning out the device cache
    gc.collect()
    torch.cuda.empty_cache()


def print_free_memory():
    free, total = torch.cuda.mem_get_info(device)
    print(f"Percent of free memory: {round(free/total *100,2)}")


empty_cache()
print_free_memory()

Percent of free memory: 98.79


Memory Summary


In [5]:
def memory_summary() -> None:
    print(torch.cuda.memory_summary())


memory_summary()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

Preparing GPU (if necessary)


In [6]:
# For AMD GPU - 7800xt
device_name = torch.cuda.get_device_name(0)
if "AMD" in device_name or "Radeon" in device_name:
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"

print(f"GPU {torch.cuda.get_device_properties(device).name} is now setup")

GPU AMD Radeon RX 7800 XT is now setup


Setting Paths for Data, Base Model, and Output Directory


In [ ]:
# Directory Paths
training_set_name, data_method = "test_dataset", "clean", 
current_directory = os.getcwd()
path_to_base_directory = re.search(rf"(.*?){"Weird-Stuff-In-Traffic"}", current_directory).group(1)
yaml_data_path = f"Weird-Stuff-In-Traffic/Data/yolo/coco8/coco8.yaml"
complete_data_path = path_to_base_directory + yaml_data_path

# Model Paths
base_model_name = "yolo11n.pt"
simple_base_model_name = base_model_name.split(".")[0]
model_results_path = f"Weird-Stuff-In-Traffic/Models/Segmentation-Detection/yolo/"
model_training_config = f"training_{training_set_name}_{data_method}_data"
model_output_dir = path_to_base_directory + model_results_path + model_training_config + "/fine_tuned_model"

Loading Model


In [8]:
model = YOLO(base_model_name)

100%|██████████| 5.35M/5.35M [00:01<00:00, 4.17MB/s]


Model Training


In [ ]:
# Train the model
model.train(
    data=complete_data_path,
    epochs=1,
    imgsz=640,
    project=model_output_dir,
    name="experiment1",                # subfolder name for this run
    batch=16,                          # batch size (adjust for your GPU)
    workers=4,                         # number of data loading workers
    device=device,
    verbose=True
)

engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=Weird-Stuff-In-Traffic/Data/yolo/coco8/coco8.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=4, project=/home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Models/Segmentation-Detection/yolo/training_test_dataset_clean_data/fine_tuned_model, name=experiment110, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

RuntimeError: Dataset 'Weird-Stuff-In-Traffic/Data/yolo/coco8/coco8.yaml' error ❌ 'Weird-Stuff-In-Traffic/Data/yolo/coco8/coco8.yaml' does not exist